<a href="https://colab.research.google.com/github/AliciaAPerez/Death_Analytics_using_CDC_Data_with_Machine_Learning/blob/Claude/Death_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Hashing").getOrCreate()

In [10]:
from pyspark import SparkFiles
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2015_data.csv"
spark.sparkContext.addFile(url)
death_2015 = spark.read.csv(SparkFiles.get("2015_data.csv"), sep=",", header=True)

# Show DataFrame
death_2015.show()

+---------------+-----------------------+-----------------------+------------------------+--------------+---+---------------+----------+---------------------+-------------+-------------+-------------+--------------------+-----------------------------------+--------------+--------------------+-----------------+--------------+---------------+---------------------+-------+-------------+------------------------------------------------------+----------------------+----------------+----------------+-----------------------+---------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------------------+-----

In [13]:
#removing unneded columns

death_2015_clean = death_2015.drop("education_1989_revision","education_reporting_flag","age_substitution_flag","age_recode_52","age_recode_27","age_recode_12", "infant_age_recode_22",
"place_of_death_and_decedents_status","method_of_disposition","autopsy", "activity_code", "place_of_injury_for_causes_w00_y34_except_y06_and_y07_", "358_cause_recode", "113_cause_recode",
"130_infant_cause_recode", "39_cause_recode", "bridged_race_flag", "race_imputation_flag", "race_recode_3", "race_recode_5", "hispanic_origin")

In [14]:
death_2015_clean.show()

+---------------+-----------------------+--------------+---+---------------+----------+--------------+--------------------+-----------------+--------------+---------------+----------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------------

In [20]:
#removing any individual under the age of one due to how their recorded in the data per the documentation
#for any subject where "detail_age_type" = 1, the detail_age is the age in years, but "detail_age_type" = 2 through 9, age is months(2), days(4), hours(5), minutes(6), or not stated (9)

death_2015_clean_adults = death_2015_clean.filter((death_2015_clean.detail_age_type == 1))
death_2015_clean_adults.show()

+---------------+-----------------------+--------------+---+---------------+----------+--------------+--------------------+-----------------+--------------+---------------+----------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------------

In [24]:
#removing anyone under the age of 1, and then removing the "detail_age_type" category as it will no longer be needed
death_2015_adults = death_2015_clean_adults.drop("detail_age_type")
death_2015_adults.show()

+---------------+-----------------------+--------------+---+----------+--------------+--------------------+-----------------+--------------+---------------+----------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------

In [28]:
death_2015_adults.printSchema()

root
 |-- resident_status: string (nullable = true)
 |-- education_2003_revision: string (nullable = true)
 |-- month_of_death: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- detail_age: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: string (nullable = true)
 |-- current_data_year: string (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: string (nullable = true)
 |-- icd_code_10th_revision: string (nullable = true)
 |-- number_of_entity_axis_conditions: string (nullable = true)
 |-- entity_condition_1: string (nullable = true)
 |-- entity_condition_2: string (nullable = true)
 |-- entity_condition_3: string (nullable = true)
 |-- entity_condition_4: string (nullable = true)
 |-- entity_condition_5: string (nullable = true)
 |-- entity_condition_6: string (nullable = true)
 |-- entity_condition_7: string (nullable = true)
 |-- entity_condition_8: string (nullable = true)
 |-- entity_

In [34]:
death_2015_adults.count()

2694400

In [56]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
death_2015_update = death_2015_adults.withColumn("number_of_entity_axis_conditions",col("number_of_entity_axis_conditions").cast("integer")).withColumn("number_of_record_axis_conditions",col("number_of_record_axis_conditions").cast("integer"))
death_2015_update.printSchema()

root
 |-- resident_status: string (nullable = true)
 |-- education_2003_revision: string (nullable = true)
 |-- month_of_death: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- detail_age: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: string (nullable = true)
 |-- current_data_year: string (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: string (nullable = true)
 |-- icd_code_10th_revision: string (nullable = true)
 |-- number_of_entity_axis_conditions: integer (nullable = true)
 |-- entity_condition_1: string (nullable = true)
 |-- entity_condition_2: string (nullable = true)
 |-- entity_condition_3: string (nullable = true)
 |-- entity_condition_4: string (nullable = true)
 |-- entity_condition_5: string (nullable = true)
 |-- entity_condition_6: string (nullable = true)
 |-- entity_condition_7: string (nullable = true)
 |-- entity_condition_8: string (nullable = true)
 |-- entity

In [59]:
#need to know on average how many underlying conditions there are 20 options are available
death_2015_update.select("number_of_entity_axis_conditions", "number_of_record_axis_conditions").describe().show()

+-------+--------------------------------+--------------------------------+
|summary|number_of_entity_axis_conditions|number_of_record_axis_conditions|
+-------+--------------------------------+--------------------------------+
|  count|                         2694400|                         2694400|
|   mean|               3.103408551068884|               2.953636059976247|
| stddev|              1.9167837941567076|              1.7928775740469587|
|    min|                               1|                               1|
|    max|                              15|                              15|
+-------+--------------------------------+--------------------------------+



In [60]:
#per documentation of the data, record_axis will be removed as it is for the death certificate, and the entity is pre-known conditions
#for the entity conditions, droppping all but the average of 3, and removing the number of entity as it is no longer needed 
death_2015_cleaned = death_2015_update.drop("number_of_entity_axis_conditions","entity_condition_4","entity_condition_5","entity_condition_6","entity_condition_7","entity_condition_8",
                                            "entity_condition_9","entity_condition_10","entity_condition_11","entity_condition_12","entity_condition_13","entity_condition_14",
                                            "entity_condition_15","entity_condition_16","entity_condition_17","entity_condition_18","entity_condition_19","entity_condition_20",
                                            "number_of_record_axis_conditions", "record_condition_1", "record_condition_2","record_condition_3","record_condition_4","record_condition_5",
                                            "record_condition_6","record_condition_7","record_condition_8","record_condition_9","record_condition_10","record_condition_11","record_condition_12",
                                            "record_condition_13","record_condition_14","record_condition_15","record_condition_16","record_condition_17","record_condition_18","record_condition_19",
                                            "record_condition_20",)

In [61]:
death_2015_cleaned.show()

+---------------+-----------------------+--------------+---+----------+--------------+--------------------+-----------------+--------------+---------------+----------------------+------------------+------------------+------------------+----+--------------------------+
|resident_status|education_2003_revision|month_of_death|sex|detail_age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_code_10th_revision|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic_originrace_recode|
+---------------+-----------------------+--------------+---+----------+--------------+--------------------+-----------------+--------------+---------------+----------------------+------------------+------------------+------------------+----+--------------------------+
|              1|                      3|            01|  M|       084|             M|                   1|             2015|             U|              7|                  I500|            11

In [63]:
death_2015_cleaned.schema.names

['resident_status',
 'education_2003_revision',
 'month_of_death',
 'sex',
 'detail_age',
 'marital_status',
 'day_of_week_of_death',
 'current_data_year',
 'injury_at_work',
 'manner_of_death',
 'icd_code_10th_revision',
 'entity_condition_1',
 'entity_condition_2',
 'entity_condition_3',
 'race',
 'hispanic_originrace_recode']

In [66]:
death_2015_columns = death_2015_cleaned.withColumnRenamed("resident_status","resident").withColumnRenamed("education_2003_revision","education").withColumnRenamed("detail_age", "age").withColumnRenamed("current_date_year", "year").withColumnRenamed("icd_code_10th_revision", "icd_death_code").withColumnRenamed("hispanic_originrace_recode", "hispanic")
death_2015_columns.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       1|        3|            01|  M|084|             M|                   1|             2015|             U|              7|          I500|            11I500|            61L031|              null|  01|       6|
|       1|        6|            01|  M|070|             M|                   2|             2015|             U|              7|          C3

In [67]:
death_2015_columns.printSchema()

root
 |-- resident: string (nullable = true)
 |-- education: string (nullable = true)
 |-- month_of_death: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: string (nullable = true)
 |-- current_data_year: string (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: string (nullable = true)
 |-- icd_death_code: string (nullable = true)
 |-- entity_condition_1: string (nullable = true)
 |-- entity_condition_2: string (nullable = true)
 |-- entity_condition_3: string (nullable = true)
 |-- race: string (nullable = true)
 |-- hispanic: string (nullable = true)



In [72]:
death_2015_int = death_2015_columns.withColumn("resident",col("resident").cast("integer")).withColumn("education",col("education").cast("integer")).withColumn("month_of_death",col("month_of_death").cast("integer")).withColumn("age",col("age").cast("integer")).withColumn("day_of_week_of_death",col("day_of_week_of_death").cast("integer")).withColumn("current_data_year",col("current_data_year").cast("integer")).withColumn("manner_of_death",col("manner_of_death").cast("integer")).withColumn("race",col("race").cast("integer")).withColumn("hispanic",col("hispanic").cast("integer"))
death_2015_int.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       1|        3|             1|  M| 84|             M|                   1|             2015|             U|              7|          I500|            11I500|            61L031|              null|   1|       6|
|       1|        6|             1|  M| 70|             M|                   2|             2015|             U|              7|          C3

In [92]:
death_2015_final = death_2015_int.fillna(value="0",subset=["entity_condition_1"]).fillna(value="0",subset=["entity_condition_2"]).fillna(value="0",subset=["entity_condition_3"])
death_2015_final.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       1|        3|             1|  M| 84|             M|                   1|             2015|             U|              7|          I500|            11I500|            61L031|                 0|   1|       6|
|       1|        6|             1|  M| 70|             M|                   2|             2015|             U|              7|          C3

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       1|        3|            01|  M|084|             M|                   1|             2015|             U|              7|          I500|            11I500|            61L031|              null|  01|       6|
|       1|        6|            01|  M|070|             M|                   2|             2015|             U|              7|          C3